In [1]:
import sys
sys.path.append('/home/rustam/EyePit/Danil_tests/MakiFlow/')
from makiflow.models import Segmentator
from makiflow.layers import *
import makiflow as mf
import tensorflow as tf
mf.set_main_gpu(2)

First we need to provide a path provider to our data that will tell the generator where to load the data from.
For this purpose we're gonna create a generator that is gonna yield strings with path to the necessary data.

In [2]:
from makiflow.models.segmentation.gen_base import PathGenerator, SegmentIterator

The path provider must inherited from the <b>PathGenerator</b> class.

In [3]:
images_path = '/mnt/data/med_data/balanced_batches/bb_danil1/images'
masks_path = '/mnt/data/med_data/balanced_batches/bb_danil1/masks'

In [4]:
from glob import glob
import os
import numpy as np


class Generator(PathGenerator):
    def __init__(self, path_imgs, path_masks):
        self.images = glob(os.path.join(path_imgs, '*.bmp'))
        self.masks = glob(os.path.join(path_masks, '*.bmp'))
        
    def next_element(self):
        while True:
            index = np.random.randint(low=0, high=len(self.images))
            
            yield {
                SegmentIterator.image: self.images[index],
                SegmentIterator.mask: self.masks[index]
            }

## Map method

The next step is to create a map method that will map from the path string to the actual data, i.e. it's a function that loads the data according the given paths.

In [5]:
# Example map method
def ex_map_method(data_paths):
    img_file = tf.read_file(data_paths[SegmentationGenerator.image])
    mask_file = tf.read_file(data_paths[SegmentationGenerator.mask])

    img = tf.image.decode_image(img_file)
    mask = tf.image.decode_image(mask_file)

    img.set_shape([1024, 1024, 3])
    mask.set_shape([1024, 1024, 3])
    
    # The mask has three channels so we need to squeeze it
    mask = mask[:, :, 0]
    
    img = tf.cast(img, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.int32)
    return img, mask

MakiFlow provides already built map methods.
<b><ol>
    <li>LoadResizeNormalize - shortcut method</li>
    <li>LoadDataMethod - constructive method (the beginning block)</li>
    <li>ResizePostMethod - constructive method</li>
    <li>NormalizePostMethod - constructive method</li>
    <li>SqueezeMaskPostMethod - constructive method</li>
    <li>ComputePositivesPostMethod - constructive method</li>
</ol></b>


Let's create a map method by combining the aforementioned ones.

In [6]:
from makiflow.models.segmentation.map_methods import LoadDataMethod, ResizePostMethod, ComputePositivesPostMethod, \
    SqueezeMaskPostMethod

In [7]:
map_method = LoadDataMethod(image_shape=[1024, 1024, 3], mask_shape=[1024, 1024, 3])

In [8]:
map_method = SqueezeMaskPostMethod()(map_method)

In [9]:
map_method = ComputePositivesPostMethod(dtype=tf.int32)(map_method)

## Generator Layer

After acquiring map method and path provider we can create the generator layer. This layer will feed the network with the data.

In [10]:
from makiflow.models.segmentation.gen_layers import InputGenLayer

In [11]:
gen_layer = InputGenLayer(
    prefetch_size=5,
    batch_size=1, 
    path_generator=Generator(images_path, masks_path),
    name='1',
    map_operation=map_method
)

## Build a model from generator

A model can be built using classic approach - layer by layer - or with the help of the builder.

In [12]:
from makiflow.save_recover import Builder

In [14]:
model = Builder.segmentator_from_json(
    json_path='/home/rustam/EyePit/Models/x65/xception_unet_v12_with_atrous.json',
    generator=gen_layer
)

Model is restored!


## Training the model

The model can trained now using familiar, but a little bit different API.
At first we need to tell the model (Segmentator) that it's gonna be trained using generator.

In [15]:
model.set_generator(gen_layer)

In [16]:
model.set_session(tf.Session())

In [18]:
model._generator.get_iterator()

{'image': <tf.Tensor 'IteratorGetNext:0' shape=(1, 1024, 1024, 3) dtype=float32>,
 'mask': <tf.Tensor 'IteratorGetNext:1' shape=(1, 1024, 1024) dtype=int32>,
 'num_positives': <tf.Tensor 'IteratorGetNext:2' shape=(1,) dtype=float32>}

Then we set an optimizer and the number of epochs as usual. But in case of training using pipelines there's no common sense for epochs, so we need to define ourselves how long the epoch is.

In [ ]:
optimizer = tf.train.AdamOptimizer()

In [ ]:
epochs = 2
iterations = 10 # How many batches are processed during one epoch

In [ ]:
model.genfit_focal(
    gamma=2.0,
    optimizer=optimizer,
    epochs=10,
    iterations=iterations
)